Data mining from PubChem 

In [10]:
import pandas as pd
import numpy as np
import urllib
import os 
import requests
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
get_ipython().magic(u'matplotlib inline')
import time
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio.PDB import *
import os, shutil
from biopandas.pdb import PandasPdb
from Bio import ExPASy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

In [29]:
# protein_query = "MAMLQTNLGFITSPTFLCPKLKVKLNSYLWFSYRSQVQKLDFSKRVNRSYKRDALLLSIKCSSSTGFDNSNVVVKEKSVSVILLAGGQGKRMKMSMPKQYIPLLGQPIALYSFFTFSRMPEVKEIVVVCDPFFRDIFEEYEESIDVDLRFAIPGKERQDSVYSGLQEIDVNSELVCIHDSARPLVNTEDVEKVLKDGSAVGAAVLGVPAKATIKEVNSDSLVVKTLDRKTLWEMQTPQVIKPELLKKGFELVKSEGLEVTDDVSIVEYLKHPVYVSQGSYTNIKVTTPDDLLLAERILSEDS"

# nucleotide_query = "ATGGCGATGCTTCAGACGAATCTTGGCTTCATTACTTCTCCGACATTTCTGTGTCCGAAGCTTAAAGTCAAATTGAACTCTTATCTGTGGTTTAGCTATCGTTCTCAAGTTCAAAAACTGGATTTTTCGAAAAGGGTTAATAGAAGCTACAAAAGAGATGCTTTATTATTGTCAATCAAGTGTTCTTCATCGACTGGATTTGATAATAGCAATGTTGTTGTGAAGGAGAAGAGTGTATCTGTGATTCTTTTAGCTGGAGGTCAAGGCAAGAGAATGAAAATGAGTATGCCAAAGCAGTACATACCACTTCTTGGTCAGCCAATTGCTTTGTATAGCTTTTTCACGTTTTCACGTATGCCTGAAGTGAAGGAAATTGTAGTTGTATGTGATCCTTTTTTCAGAGACATTTTTGAAGAATACGAAGAATCAATTGATGTTGATCTTAGATTCGCTATTCCTGGCAAAGAAAGACAAGATTCTGTTTACAGTGGACTTCAGGAAATCGATGTGAACTCTGAGCTTGTTTGTATCCACGACTCTGCCCGACCATTGGTGAATACTGAAGATGTCGAGAAGGTCCTTAAAGATGGTTCCGCGGTTGGAGCAGCTGTACTTGGTGTTCCTGCTAAAGCTACAATCAAAGAGGTCAATTCTGATTCGCTTGTGGTGAAAACTCTCGACAGAAAAACCCTATGGGAAATGCAGACACCACAGGTGATCAAACCAGAGCTATTGAAAAAGGGTTTCGAGCTTGTAAAAAGTGAAGGTCTAGAGGTAACAGATGACGTTTCGATTGTTGAATACCTCAAGCATCCAGTTTATGTCTCTCAAGGATCTTATACAAACATCAAGGTTACAACACCTGATGATTTACTGCTTGCTGAGAGAATCTTGAGCGAGGACTCATGA"

query = Seq(input("Paste input in a form of FASTA sequence: "))

# def blast_input():
# my_query = Seq(nucleotide_query)
# query = SeqIO.read("sample.fasta", format="fasta")
try:
    print("Query sent to NCBIWWW against swissprot database")
    result_handle = NCBIWWW.qblast("blastp", "swissprot", query)
    blast_result = open("my_blast_result.xml", "w")
    blast_result.write(result_handle.read())
    blast_result.close()
    result_handle.close()
except ValueError:
    my_translated_query = my_query.translate()
    print(f"DNA/RNA sequence detected. Sequence translated to : {my_translated_query}")
    result_handle = NCBIWWW.qblast("blastp", "swissprot", my_translated_query)
    blast_result = open("my_blast_result.xml", "w")
    blast_result.write(result_handle.read())
    blast_result.close()
    result_handle.close()


# def read_blast_output():
result = open("my_blast_result.xml")
blast_records = NCBIXML.parse(result)
number_of_alignments = 0
hit_ids = []
for rec in blast_records:
    for alignment in rec.alignments:
        hit_ids.append(alignment.accession)
        number_of_alignments += 1
        if number_of_alignments == 3: # Take three top alignments and stop
            break
print(f"Top hits: {hit_ids}")
# return hit_ids
if len(hit_ids) == 0:
    print("No hits identified in the database. Check your FASTA input.")
# def read_target_db(input_list: list):
with open('target', 'r') as df:
    for line in df:
        line = line.split()
        try:
            for entry in hit_ids:
                if entry in line[2]:
                    print(f"Entry {entry} present in database!")
                    check_id = entry
                    break
        except:
            print("Target not found in database")

# if __name__ == '__main__':
#     blast_input()
#     read_target_db(read_blast_output())

Query sent to NCBIWWW against swissprot database
Top hits: ['P69834', 'Q5N8G1', 'Q6MEE8']
Target not found in database
Target not found in database
Entry P69834 present in database!


In [3]:
link1 = "https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=csv&query={%22download%22:%22*%22,%22collection%22:%22bioactivity%22,%22where%22:{%22ands%22:[{%22protacxn%22:%22notnull%22},{%22cid%22:%22notnull%22},{%22repacxn%22:%22P0C6U8%22}]},%22order%22:[%22activity,asc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22{PROTACXN_P0C6U8}_bioactivity_protein%22}"

In [4]:
link1 = link1.replace('PROTACXN_P0C6U8', check_id)
link1


'https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=csv&query={%22download%22:%22*%22,%22collection%22:%22bioactivity%22,%22where%22:{%22ands%22:[{%22protacxn%22:%22notnull%22},{%22cid%22:%22notnull%22},{%22repacxn%22:%22P0C6U8%22}]},%22order%22:[%22activity,asc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22{P69834}_bioactivity_protein%22}'

In [5]:
for i in range(0,2):
    try:
        os.remove('downloaded1.csv')
        #print("Deleted old File")
        break
    except Exception as e:
        #print("No file ")
        break
    else:
        break

In [6]:
for i in range(2):
    try:
        data = pd.read_csv(link1)
        break
    #except IncompleteRead as I:
     #   print("Server Overloading , Proceeding")
      #  break
    except Exception as a:
        print(str(a)+" is the error , Trying {} time".format(i))
        continue
    else:
        break
else:
    print("something Wrong , Try running Again [refer error code for more]")

In [ ]:
#data = pd.read_csv("downloaded1.csv",error_bad_lines=False)

In [7]:
data = pd.DataFrame(data)
data

,baid,activity,aid,sid,mid,cid,geneid,taxid,pmid,aidtype,...,acname,acvalue,aidsrcname,aidname,cmpdname,targetname,targeturl,dois,ecs,repacxn
0,98754968,Active,241511,103458247,0,11667869,NaN,227859,15896959,Confirmatory,...,IC50,0.98,ChEMBL,In vitro inhibitory concentration SARS coronav...,1-Benzo[b]thiophen-2-ylmethyl-7-bromo-1H-indol...,Replicase polyprotein 1a (SARS coronavirus),/protein/P0C6U8,10.1016/j.bmcl.2005.04.027,"3.4.19.12,3.4.22.-,3.4.22.69",P0C6U8
1,98758767,Active,241749,103458062,0,44398002,NaN,227859,15896959,Confirmatory,...,IC50,9.40,ChEMBL,In vitro inhibitory concentration against SARS...,1-(2-Chloro-4-fluoro-benzyl)-5-iodo-1H-indole-...,Replicase polyprotein 1a (SARS coronavirus),/protein/P0C6U8,10.1016/j.bmcl.2005.04.027,"3.4.19.12,3.4.22.-,3.4.22.69",P0C6U8
2,98758768,Active,241749,103458580,0,44398207,NaN,227859,15896959,Confirmatory,...,IC50,4.82,ChEMBL,In vitro inhibitory concentration against SARS...,1-Benzo[b]thiophen-2-ylmethyl-5-fluoro-1H-indo...,Replicase polyprotein 1a (SARS coronavirus),/protein/P0C6U8,10.1016/j.bmcl.2005.04.027,"3.4.19.12,3.4.22.-,3.4.22.69",P0C6U8
3,98758769,Active,241749,103458848,0,44398343,NaN,227859,15896959,Confirmatory,...,IC50,2.00,ChEMBL,In vitro inhibitory concentration against SARS...,1-Benzo[b]thiophen-2-ylmethyl-7-nitro-1H-indol...,Replicase polyprotein 1a (SARS coronavirus),/protein/P0C6U8,10.1016/j.bmcl.2005.04.027,"3.4.19.12,3.4.22.-,3.4.22.69",P0C6U8
4,98758770,Active,241749,103459046,0,44398436,NaN,227859,15896959,Confirmatory,...,IC50,17.50,ChEMBL,In vitro inhibitory concentration against SARS...,5-Iodo-1-[[5-(piperidine-1-carbonyl)thiophen-2...,Replicase polyprotein 1a (SARS coronavirus),/protein/P0C6U8,10.1016/j.bmcl.2005.04.027,"3.4.19.12,3.4.22.-,3.4.22.69",P0C6U8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,424668663,Unspecified,688306,163312872,0,14610613,NaN,227859,22884354,Literature-derived,...,NaN,NaN,ChEMBL,Time dependent inhibition of SARS-CoV PLpro ex...,Methyl tanshinonate,Replicase polyprotein 1a (SARS coronavirus),/protein/P0C6U8,10.1016/j.bmc.2012.07.038,"3.4.19.12,3.4.22.-,3.4.22.69",P0C6U8
337,424668664,Unspecified,688306,103456666,0,164676,NaN,227859,22884354,Literature-derived,...,NaN,NaN,ChEMBL,Time dependent inhibition of SARS-CoV PLpro ex...,Tanshinone IIA,Replicase polyprotein 1a (SARS coronavirus),/protein/P0C6U8,10.1016/j.bmc.2012.07.038,"3.4.19.12,3.4.22.-,3.4.22.69",P0C6U8
338,424668665,Unspecified,688306,103521465,0,11425923,NaN,227859,22884354,Literature-derived,...,NaN,NaN,ChEMBL,Time dependent inhibition of SARS-CoV PLpro ex...,Dihydrotanshinone I,Replicase polyprotein 1a (SARS coronavirus),/protein/P0C6U8,10.1016/j.bmc.2012.07.038,"3.4.19.12,3.4.22.-,3.4.22.69",P0C6U8
339,424668696,Unspecified,688308,103227294,0,160142,NaN,227859,22884354,Confirmatory,...,IC50,78.90,ChEMBL,Inhibition of SARS-CoV PLpro expressed in Esch...,Miltirone,Replicase polyprotein 1a (SARS coronavirus),/protein/P0C6U8,10.1016/j.bmc.2012.07.038,"3.4.19.12,3.4.22.-,3.4.22.69",P0C6U8


In [8]:
#data cleaning up
new_data = data[['cid','acvalue']]
new_data = new_data.dropna()

In [9]:
except_val=0
cid_value = new_data['cid'].to_list()
PIC50_value = (-np.log10(new_data['acvalue']*10**-6)).to_list()

PIC50_value = pd.DataFrame(PIC50_value,columns = ["y"])
y_data = PIC50_value

In [10]:
#WebAPI PubChem

link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/new_link_me/property/MolecularWeight,HeavyAtomCount,XLOGP,Complexity,HBondAcceptorCount,MonoisotopicMass,RotatableBondCount,TPSA/CSV"
link_fixed = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/new_link_me/property/MolecularWeight,HeavyAtomCount,XLOGP,Complexity,HBondAcceptorCount,MonoisotopicMass,RotatableBondCount,TPSA/CSV"
sub_link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastsimilarity_2d/cid/replaceme/cids/TXT"
sub_link_fixed = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastsimilarity_2d/cid/replaceme/cids/TXT"
#the link with the required properties.
counter = 1

In [ ]:
final_data_frame = pd.DataFrame()
x_data = pd.DataFrame()
cid_main_counter=0 
for x in cid_value:
    for iter_x in range(1000):
        try:    
            link = link_fixed
            link = link.replace("new_link_me",str(x))
            data1 = pd.read_csv(link)
            data1 = pd.DataFrame(data1)
            x_data = x_data.append(data1)
            data1 = 0
        except Exception as e:
            print("Exception Encountered as {} .! , Trying again !Iteration : {}".format(str(e),str(iter_x)))
            time.sleep(5)
            continue
        else:
            break
    
    else:
        print("Something Wrong with the Trials ! Restart The Algorithm!")

In [ ]:
x_data_saved = x_data
x_data = x_data_saved
n_cid=[]

In [ ]:
#20 sec Wait_time
for i in cid_value:
    max_tries = 10
    
    for iter_ in range(max_tries):
        try:
            #print("inside Try")
            #time.sleep(5)


            link = link_fixed
            link = link.replace("new_link_me",str(i))
            f_data = pd.read_csv(link) 
            
            sub_link = sub_link_fixed
            sub_link = sub_link.replace("replaceme",str(i))
            res = urllib.request.urlopen(sub_link)
            data_sub = res.read()
            data_sub = str(data_sub)
            data_sub = data_sub.replace("\\n",",") ; data_sub = data_sub.replace('b'," ")
            data_sub = data_sub.replace("'",""); data_sub = data_sub.replace(" ","")
            n_count_loop = 0 
            #time.sleep(3)
            #print("Going for the loop")
            for j in data_sub.split(","):
                n_count_loop+=1
                if(len(j)>1):
                    if(n_count_loop<=30):
                        n_cid.append(j)
                    else:
                        break
                else:
                    pass
             
        except:
            if(except_val>=15):
                break
            else:
                print("Re-Trying")
                except_val+=1
                #time.sleep(3)
                continue
        else:
            break
    '''else:
        print("Either The Network is Down(So no point in continuing ) , Or Some Uknown Error Spotted!. Refer Error Code , Continuing with Fetched Data")
        time.sleep(5)
        break'''
    
    
    '''final_data_frame = final_data_frame.append(f_data)
    cid_main_counter+=1
    final_data_frame.to_csv("fdf1.csv")
         
   

    print("Main cid no: {}".format(cid_main_counter))
    #time.sleep(5)'''
    



In [ ]:
k_count = 0
t_count = 0
phase_count=0
f_data = pd.DataFrame()

In [ ]:
#data mining PubChem through web API

for k in n_cid:
    k_count+=1
    t_count+=1
    phase_count+=1
    for sub_iter in range(10):
            try:
                if(t_count>=50):
                    #print("taking Rest")
                    time.sleep(15)
                    t_count=0
                    
                #print("Inside Try"+str(k_count))
            
                link = link_fixed
                link = link.replace("new_link_me",str(k))
                #print("replacement done"+str(k_count))
                f_data_df = pd.read_csv(link)
                f_data = f_data.append(f_data_df)
                print("Passed Without Exception"+str(k_count))
                break

            except Exception as e:
                print(str(e) + "Encountered , Please Wait :+ "+str(k_count))
                time.sleep(20)
                continue

    final_data_frame = final_data_frame.append(f_data)
    cid_main_counter+=1
    final_data_frame.to_csv("fdf1.csv")
    print("Data Fetching Continued"+ str(k_count))
    
    


In [ ]:
final_data_frame.drop_duplicates(inplace=True)
print(final_data_frame.shape)
print("The Final Training DataSet: X_data = {} , Y_data = {}".format(x_data.shape,y_data.shape))
print("The Final Data For Preditcion is : {}".format(final_data_frame.shape))

In [ ]:
try:
    final_data_frame.reset_index(inplace=True)
    final_data_frame.drop('index',axis=1,inplace=True)
except:
    pass


In [ ]:
final_data_frame_reset = final_data_frame
x_data_reset = x_data
y_data_reset = y_data

Auto QSAR algorithm

In [ ]:
'''#RUN TO RESET ONLY : TESTING CAUTION
final_data_frame = final_data_frame_reset
x_data = x_data_reset
y_data = y_data_reset'''

In [ ]:
#data preparation for QSAR model building
x_data = pd.DataFrame(x_data)

In [ ]:
y_data = pd.DataFrame(y_data)

In [ ]:
print("X :- "+str(x_data.shape)+" Y :- "+str(y_data.shape) )

In [ ]:
train_file = pd.DataFrame()
x_data = x_data.astype("float64")
y_data = y_data.astype("float64")

In [ ]:
#molecular features for AutoQSAR model building
x_data.head()

In [ ]:
#experimental inhibition value for model building
y_data.head()

In [ ]:
x_data.head()

In [ ]:
new = y_data['y'].to_list()

In [ ]:
train_file  = x_data
train_file['y'] = new

In [ ]:
cid_reg_list=train_file['CID'].to_list()
train_file.drop('CID',axis=1,inplace =True)


In [ ]:
for x in train_file.isnull().any():
    if x == True:
        train_file = train_file.fillna(method='ffill')
train_file

#data set of molecular features and experimental inhibition value for QSAR model building

In [ ]:
x = list(train_file.columns)
x = x[:-1]

x_ = train_file[x].values
y_ = train_file['y'].values

In [ ]:
train_file.describe()
x_ = x_.astype("float64")
y_ = y_.astype("float64")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size=0.1, random_state=0)


In [ ]:
#AutoQSAR workflow

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

In [ ]:
scaler = preprocessing.StandardScaler()
degree=1
polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
polyreg_scaled.fit(X_train,y_train)

In [ ]:
#R2 Score
y_pred = polyreg_scaled.predict(X_test)
main_r2 = r2_score(y_test, y_pred, multioutput='uniform_average')
max_ = main_r2 
max_

In [ ]:
from itertools import combinations
comb_list = [[]]

In [ ]:
#creating combinations , function
def sub(arr,r):
    global comb_list
    for i in r:
        comb = list(combinations(arr,i))
        comb_list.append(comb)
    return comb_list
newone = 0
newone = sub(x , [2,3,4,5,6])
del newone[0]

In [ ]:
#QSAR model building based on linear multivariate regression for all possible combinations of descriptors and choosing the model with best statistical quality 

In [ ]:
#degree 1 
coef_dict  = 0
loop_index = 0
coef_dict = [[]]
r2_score_new =[]
max_r2 = []
index_r2 =[]
for i in range(0,len(newone)):
    
    index_r=0
    for combi_ in newone[loop_index]:
        
        #print(combi_)
        features = list(combi_)
        features_=train_file[features].values
        output_=train_file['y'].values
        X_train, X_test, y_train, y_test = train_test_split(features_, output_, test_size=0.1, random_state=0)
        scaler = preprocessing.StandardScaler()
        degree=1
        polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
        polyreg_scaled.fit(X_train,y_train)
        y_pred = polyreg_scaled.predict(X_test)
        r2_score_  = r2_score(y_test, y_pred, multioutput='uniform_average')
        r2_score_new.append(r2_score_)
    loop_index+=1
        
    
    max_r2.append(max(r2_score_new))
    index_r = r2_score_new.index(max(r2_score_new))
    index_r2.append(index_r)
    r2_score_new =[]

In [ ]:
#Extracting the best value so far using the regressor
sec_index = max_r2.index(max(max_r2))
fir_index = index_r2[sec_index]


In [ ]:
if main_r2 > max(max_r2):
    r2_features = x
else:
    
    features  = newone[sec_index][fir_index]
    maxi_r2 = max(max_r2)

In [ ]:
print("R2 - degree1 = " + str(maxi_r2)+str("\n")+str("Features = ")+str(features))
reg_max_r2 = max(max_r2)

In [ ]:
#We have the best degree 1 model
model_dict = {}
model_dict[1]=[maxi_r2,features]

In [ ]:
for x in train_file.isnull().any():
    if x == True:
        train_file = train_file.fillna(method='ffill')

In [ ]:
#QSAR model building based on non-linear multivariate regression(degree 2) for all possible combinations of descriptors and choosing the model with best statistical quality 
coef_dict  = 0
loop_index = 0
coef_dict = [[]]
r2_score_new =[]
max_r2 = []
index_r2 =[]
for i in range(0,len(newone)):
    
    index_r=0
    for combi_ in newone[loop_index]:
        
        #print(combi_)
        features = list(combi_)
        features_=train_file[features].values
        output_=train_file['y'].values
        X_train, X_test, y_train, y_test = train_test_split(features_, output_, test_size=0.1, random_state=0)
        scaler = preprocessing.StandardScaler()
        degree=2
        polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
        polyreg_scaled.fit(X_train,y_train)
        y_pred = polyreg_scaled.predict(X_test)
        r2_score_  = r2_score(y_test, y_pred, multioutput='uniform_average')
        r2_score_new.append(r2_score_)
    loop_index+=1
        
    
    max_r2.append(max(r2_score_new))
    index_r = r2_score_new.index(max(r2_score_new))
    index_r2.append(index_r)
    r2_score_new =[]

In [ ]:
#Extracting the best value so far using the regressor
sec_index = max_r2.index(max(max_r2))
fir_index = index_r2[sec_index]

In [ ]:
if main_r2 > max(max_r2):
    r2_features = x
else:
    
    features  = newone[sec_index][fir_index]
    maxi_r2 = max(max_r2)

In [ ]:
print("R2 - degree2 = " + str(maxi_r2)+str("\n")+str("Features = ")+str(features))
reg_max_r2 = max(max_r2)

In [ ]:
model_dict[2]=[maxi_r2,features]

In [ ]:
for x in train_file.isnull().any():
    if x == True:
        train_file = train_file.fillna(method='ffill')

In [ ]:
model_dict

In [ ]:
#QSAR model building based on non-linear multivariate regression(degree 3) for all possible combinations of descriptors and choosing the model with best statistical quality 

Data mining from PubChem

import pandas as pd

import numpy as np

import urllib

import os 

import requests

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

import seaborn as seabornInstance 

from sklearn.model_selection import train_test_split 

from sklearn.linear_model import LinearRegression

from sklearn import metrics

get_ipython().magic(u'matplotlib inline')

import time

link1 = "https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=csv&query={%22download%22:%22*%22,%22collection%22:%22bioactivity%22,%22where%22:{%22ands%22:[{%22protacxn%22:%22notnull%22},{%22cid%22:%22notnull%22},{%22repacxn%22:%22P0C6U8%22}]},%22order%22:[%22activity,asc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22PROTACXN_P0C6U8_bioactivity_protein%22}"

for i in range(0,2):

    try:

        os.remove('downloaded1.csv')

        #print("Deleted old File")

        break

    except Exception as e:

        #print("No file ")

        break

    else:

        break

for i in range(2):

    try:

        data = pd.read_csv(link1)

        break

    #except IncompleteRead as I:

     #   print("Server Overloading , Proceeding")

      #  break

    except Exception as a:

        print(str(a)+" is the error , Trying {} time".format(i))

        continue

    else:

        break

else:

    print("something Wrong , Try running Again [refer error code for more]")

#data = pd.read_csv("downloaded1.csv",error_bad_lines=False)

data = pd.DataFrame(data)

data

	baid 	activity 	aid 	sid 	mid 	cid 	geneid 	taxid 	pmid 	aidtype 	... 	acname 	acvalue 	aidsrcname 	aidname 	cmpdname 	targetname 	targeturl 	dois 	ecs 	repacxn
0 	98754968 	Active 	241511 	103458247 	0 	11667869 	NaN 	227859 	15896959 	Confirmatory 	... 	IC50 	0.98 	ChEMBL 	In vitro inhibitory concentration SARS coronav... 	1-Benzo[b]thiophen-2-ylmethyl-7-bromo-1H-indol... 	Replicase polyprotein 1a (SARS coronavirus) 	/protein/P0C6U8 	10.1016/j.bmcl.2005.04.027 	3.4.19.12,3.4.22.-,3.4.22.69 	P0C6U8
1 	98758767 	Active 	241749 	103458062 	0 	44398002 	NaN 	227859 	15896959 	Confirmatory 	... 	IC50 	9.40 	ChEMBL 	In vitro inhibitory concentration against SARS... 	1-(2-Chloro-4-fluoro-benzyl)-5-iodo-1H-indole-... 	Replicase polyprotein 1a (SARS coronavirus) 	/protein/P0C6U8 	10.1016/j.bmcl.2005.04.027 	3.4.19.12,3.4.22.-,3.4.22.69 	P0C6U8
2 	98758768 	Active 	241749 	103458580 	0 	44398207 	NaN 	227859 	15896959 	Confirmatory 	... 	IC50 	4.82 	ChEMBL 	In vitro inhibitory concentration against SARS... 	1-Benzo[b]thiophen-2-ylmethyl-5-fluoro-1H-indo... 	Replicase polyprotein 1a (SARS coronavirus) 	/protein/P0C6U8 	10.1016/j.bmcl.2005.04.027 	3.4.19.12,3.4.22.-,3.4.22.69 	P0C6U8
3 	98758769 	Active 	241749 	103458848 	0 	44398343 	NaN 	227859 	15896959 	Confirmatory 	... 	IC50 	2.00 	ChEMBL 	In vitro inhibitory concentration against SARS... 	1-Benzo[b]thiophen-2-ylmethyl-7-nitro-1H-indol... 	Replicase polyprotein 1a (SARS coronavirus) 	/protein/P0C6U8 	10.1016/j.bmcl.2005.04.027 	3.4.19.12,3.4.22.-,3.4.22.69 	P0C6U8
4 	98758770 	Active 	241749 	103459046 	0 	44398436 	NaN 	227859 	15896959 	Confirmatory 	... 	IC50 	17.50 	ChEMBL 	In vitro inhibitory concentration against SARS... 	5-Iodo-1-[[5-(piperidine-1-carbonyl)thiophen-2... 	Replicase polyprotein 1a (SARS coronavirus) 	/protein/P0C6U8 	10.1016/j.bmcl.2005.04.027 	3.4.19.12,3.4.22.-,3.4.22.69 	P0C6U8
... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	... 	...
336 	424668663 	Unspecified 	688306 	163312872 	0 	14610613 	NaN 	227859 	22884354 	Literature-derived 	... 	NaN 	NaN 	ChEMBL 	Time dependent inhibition of SARS-CoV PLpro ex... 	Methyl tanshinonate 	Replicase polyprotein 1a (SARS coronavirus) 	/protein/P0C6U8 	10.1016/j.bmc.2012.07.038 	3.4.19.12,3.4.22.-,3.4.22.69 	P0C6U8
337 	424668664 	Unspecified 	688306 	103456666 	0 	164676 	NaN 	227859 	22884354 	Literature-derived 	... 	NaN 	NaN 	ChEMBL 	Time dependent inhibition of SARS-CoV PLpro ex... 	
coef_dict  = 0
loop_index = 0
coef_dict = [[]]
r2_score_new =[]
max_r2 = []
index_r2 =[]
for i in range(0,len(newone)):
    
    index_r=0
    for combi_ in newone[loop_index]:
        
        #print(combi_)
        features = list(combi_)
        features_=train_file[features].values
        output_=train_file['y'].values
        X_train, X_test, y_train, y_test = train_test_split(features_, output_, test_size=0.1, random_state=0)
        scaler = preprocessing.StandardScaler()
        degree=3
        polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
        polyreg_scaled.fit(X_train,y_train)
        y_pred = polyreg_scaled.predict(X_test)
        r2_score_  = r2_score(y_test, y_pred, multioutput='uniform_average')
        r2_score_new.append(r2_score_)
    loop_index+=1
        
    
    max_r2.append(max(r2_score_new))
    index_r = r2_score_new.index(max(r2_score_new))
    index_r2.append(index_r)
    r2_score_new =[]

In [ ]:
#Extracting the best value so far using the regressor
sec_index = max_r2.index(max(max_r2))
fir_index = index_r2[sec_index]

In [ ]:
if main_r2 > max(max_r2):
    r2_features = x
else:
    
    features  = newone[sec_index][fir_index]
    maxi_r2 = max(max_r2)

In [ ]:
print("R2 - degree3 = " + str(maxi_r2)+str("\n")+str("Features = ")+str(features))
reg_max_r2 = max(max_r2)

In [ ]:
model_dict[3]=[maxi_r2,features]

In [ ]:
for x in train_file.isnull().any():
    if x == True:
        train_file = train_file.fillna(method='ffill')

In [ ]:
model_dict

In [ ]:
top_cid_mix_df = pd.DataFrame()

In [ ]:
#Prediction with the validated model
model_dict[1]

In [ ]:
features = list(model_dict[1][1])
features

In [ ]:
x_trained =train_file[features].values
y_trained = train_file['y'].values
x_trained.shape,y_trained.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_trained, y_trained, test_size=0.1, random_state=0)

In [ ]:
#poly regression - degree 1
scaler = preprocessing.StandardScaler()
degree=1
polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
polyreg_scaled.fit(X_train,y_train)

In [ ]:
#R2 Score
y_pred = polyreg_scaled.predict(X_test)
r2_score(y_test, y_pred, multioutput='uniform_average')

In [ ]:
final_data_frameF1 = final_data_frame[final_data_frame['MolecularWeight'] <= 500]
final_data_frameF2 = final_data_frameF1[final_data_frameF1['XLogP'] <=5.0]

In [ ]:
pred_data_cid = final_data_frameF2["CID"].to_list()

In [ ]:
#features.append('CID')
new_features = features.append("CID")
features

In [ ]:
final_data_frame1 = final_data_frameF2[features]
for x in final_data_frame1.isnull().any():
    if x == True:
        final_data_frame1 = final_data_frame1.fillna(method='ffill')
final_data_frame1.drop(columns='CID',inplace=True)
final_data_frame1.shape

In [ ]:
final_pred_1 = polyreg_scaled.predict(final_data_frame1)

In [ ]:
final_pred_1 = list(final_pred_1)
final_data_frame1['CID'] = pred_data_cid
final_data_frame1['y_'] = final_pred_1

In [ ]:
saved_final_data_frame1 = final_data_frame1
sorted_final_df = final_data_frame1.sort_values('y_',ascending =0)
final_larg= sorted_final_df.head(50)

In [ ]:
final_larg = final_larg[['CID','y_']]
top_cid = final_larg["CID"].to_list()

In [ ]:
print("The Top 50 Drug Leads Which are identified with PubChem cid's are (USING Linear REGRESSION): ")
itter_count = 0
for itter in top_cid:
    itter_count+=1
    
    print(str(itter_count)+" : "+str(itter))

In [ ]:
print("r2 = "+str(r2_score(y_test, y_pred, multioutput='uniform_average')))

In [ ]:
top_cid_mix_df['Degree 1'] = top_cid

In [ ]:
final_cid_1 = pd.DataFrame()
final_cid_1['CID'] = top_cid
final_cid_1.to_csv("final_cid_degree_1.csv")

In [ ]:
model_dict[2]

In [ ]:
features = list(model_dict[2][1])
features

In [ ]:
x_trained =train_file[features].values
y_trained = train_file['y'].values
x_trained.shape,y_trained.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_trained, y_trained, test_size=0.1, random_state=0)

In [ ]:
#poly regression - degree 2
scaler = preprocessing.StandardScaler()
degree=2
polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
polyreg_scaled.fit(X_train,y_train)

In [ ]:
#R2 Score
y_pred = polyreg_scaled.predict(X_test)
r2_score(y_test, y_pred, multioutput='uniform_average')

In [ ]:
final_data_frameF1 = final_data_frame[final_data_frame['MolecularWeight'] <= 500]
final_data_frameF2 = final_data_frameF1[final_data_frameF1['XLogP'] <=5.6]

In [ ]:
pred_data_cid = final_data_frameF2["CID"].to_list()

In [ ]:
#features.append('CID')
new_features = features.append("CID")
features

In [ ]:
final_data_frame1 = final_data_frameF2[features]
for x in final_data_frame1.isnull().any():
    if x == True:
        final_data_frame1 = final_data_frame1.fillna(method='ffill')
final_data_frame1.drop(columns='CID',inplace=True)
final_data_frame1.shape

In [ ]:
final_pred_1 = polyreg_scaled.predict(final_data_frame1)
final_pred_1 = list(final_pred_1)
final_data_frame1['CID'] = pred_data_cid
final_data_frame1['y_'] = final_pred_1

In [ ]:
saved_final_data_frame1 = final_data_frame1
sorted_final_df = final_data_frame1.sort_values('y_',ascending =0)
final_larg= sorted_final_df.head(50)

In [ ]:
final_larg = final_larg[['CID','y_']]
top_cid = final_larg["CID"].to_list()

In [ ]:
print("The Top 50 Drug Leads Which are identified with PubChem cid's are (USING DEGREE 2 REGRESSION): ")
itter_count = 0
for itter in top_cid:
    itter_count+=1
    
    print(str(itter_count)+" : "+str(itter))

In [ ]:
top_cid_mix_df['Degree 2'] = top_cid

In [ ]:
print("r2 = "+str(r2_score(y_test, y_pred, multioutput='uniform_average')))
final_cid_2 = pd.DataFrame()
final_cid_2['CID'] = top_cid
final_cid_2.to_csv("final_cid_degree_2.csv")

In [ ]:
model_dict[3]

In [ ]:
features = list(model_dict[3][1])
features

In [ ]:
x_trained =train_file[features].values
y_trained = train_file['y'].values
x_trained.shape,y_trained.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_trained, y_trained, test_size=0.1, random_state=0)

In [ ]:
#poly regression - degree 3
scaler = preprocessing.StandardScaler()
degree=3
polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
polyreg_scaled.fit(X_train,y_train)

In [ ]:
#R2 Score
y_pred = polyreg_scaled.predict(X_test)
r2_score(y_test, y_pred, multioutput='uniform_average')

In [ ]:
final_data_frameF1 = final_data_frame[final_data_frame['MolecularWeight'] <= 500]
final_data_frameF2 = final_data_frameF1[final_data_frameF1['XLogP'] <=5.6]

In [ ]:
pred_data_cid = final_data_frameF2["CID"].to_list()

In [ ]:
#features.append('CID')
new_features = features.append("CID")
features

In [ ]:
final_data_frame1 = final_data_frameF2[features]
for x in final_data_frame1.isnull().any():
    if x == True:
        final_data_frame1 = final_data_frame1.fillna(method='ffill')
final_data_frame1.drop(columns='CID',inplace=True)
final_data_frame1.shape

In [ ]:
final_pred_1 = polyreg_scaled.predict(final_data_frame1)
final_pred_1 = list(final_pred_1)
final_data_frame1['CID'] = pred_data_cid
final_data_frame1['y_'] = final_pred_1

In [ ]:
saved_final_data_frame1 = final_data_frame1
sorted_final_df = final_data_frame1.sort_values('y_',ascending =0)
final_larg= sorted_final_df.head(50)

In [ ]:
final_larg = final_larg[['CID','y_']]
top_cid = final_larg["CID"].to_list()

In [ ]:
print("The Top 50 Drug Leads Which are identified with PubChem cid's are (USING DEGREE 3 REGRESSION): ")
itter_count = 0
for itter in top_cid:
    itter_count+=1
    
    print(str(itter_count)+" : "+str(itter))

In [ ]:
top_cid_mix_df['Degree 3'] = top_cid

In [ ]:
print("r2 = "+str(r2_score(y_test, y_pred, multioutput='uniform_average')))
final_cid_2 = pd.DataFrame()
final_cid_2['CID'] = top_cid
final_cid_2.to_csv("final_cid_degree_2.csv")

 THE END - COMPOUND FETCHING

In [ ]:
top_cid_mix_df

In [ ]:
top_cid_mix_df.to_csv("TOP_CID_123.csv")



top_cid = top_cid_mix_df['Degree 3']

top_cid

Automated In Silico modelling 

In [75]:
class ResSelect(Select):
    def accept_residue(self, residue):
        if is_aa(residue):
            return 1
        else:
            return 0


class ProteinPreparer:



    def __init__(self, protein: str, dir: str):
        self.protein = protein
        self.tmpdir = dir
        self.filename = self.tmpdir + '/pdb{}.ent'.format(self.protein)

    def prepare_protein(self):
        parser = PDBParser()
        io = PDBIO()
        PDBList().retrieve_pdb_file(pdb_code=self.protein, pdir=self.tmpdir,
                                    file_format='pdb')  # saves pdb in a form of ent file
        ipdb = parser.get_structure('ipdb', self.filename)  # Input pdb as a self.filename
        io.set_structure(ipdb)  # Setting structure for input pdb
        io.save(self.tmpdir + '/'+self.protein + '.pdb', ResSelect(),
                preserve_atom_numbering=True)  # saves the cleaned pdb
        os.system(prepare_protein_path + self.tmpdir + '/{}.pdb'.format(
            self.protein))  # prepares the structure by adding polar hydrogens and adding gesteiger charges
        shutil.move(self.protein + '.pdbqt', self.tmpdir)

prepare_ligand_path = '~/MGLTools-1.5.6/bin/pythonsh ~/MGLTools-1.5.6/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py -A bonds_hydrogens -U nphs_lps -l'

class LigandPreparer:

    def __init__(self, ligand_file: str, dir: str):
        self.ligand = ligand_file
        self.tmpdir = dir

    def prepare_ligand(self):
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/SDF'.format(self.ligand)
        urllib.request.urlretrieve(url, self.tmpdir + '/{}'.format(self.ligand + '.sdf')) # downloads the file
        os.system('obabel {} -O {} --gen3d'.format(
            self.tmpdir + '/' + self.ligand + '.sdf', self.tmpdir + '/' + self.ligand + 'prep.pdb')
        ) # generates 3d coords
        os.system(
            'obminimize -ff GAFF {} > {}'.format(
                self.tmpdir + '/' + self.ligand + 'prep.pdb', self.tmpdir + '/' + self.ligand + '.pdb'
                                                 )
        ) # minimizes using GAFF
        os.system(prepare_ligand_path + '{}'.format(self.tmpdir + '/' + self.ligand + '.pdb'
                                                    )) # adds charges, sets rotatable bonds
        shutil.move(self.ligand + '.pdbqt', self.tmpdir)

class VinaDocker:

    def __init__(self, ligentry: str, protentry: str, protein_pdbqt: str, ligand_pdbqt: str, dir: str):
        self.protein = protein_pdbqt + '.pdbqt'
        self.protpdb = protentry + '.pdb'
        self.protname = os.path.basename(self.protein)
        self.ligand = ligand_pdbqt + '.pdbqt'
        self.ligname = os.path.basename(self.ligand)
        self.tmpdir = dir
        # self.docklog = './results/' + protentry + '_' + ligentry + '_docking.log' # !!
        # self.dockfile =  './results/' + protentry + '_' + ligentry + '.out' # !!
        self.docklog = protentry + '_' + ligentry + '_docking.log' # !!
        self.dockfile = protentry + '_' + ligentry + '.out' # !!
        self.complex_name = protentry + '_' + ligentry + '_cplx.pdb'

    def dock_merge_plip(self):
        df = PandasPdb().read_pdb(self.tmpdir + '/' + self.protpdb)  # opens protein to calculate grid
        minx = df.df['ATOM']['x_coord'].min()
        maxx = df.df['ATOM']['x_coord'].max()
        cent_x = round((maxx + minx) / 2, 2)
        size_x = round(abs(maxx - minx) + 3, 2)
        miny = df.df['ATOM']['y_coord'].min()
        maxy = df.df['ATOM']['y_coord'].max()
        cent_y = round((maxy + miny) / 2, 2)
        size_y = round(abs(maxy - miny) + 3, 2)
        minz = df.df['ATOM']['z_coord'].min()
        maxz = df.df['ATOM']['z_coord'].max()
        cent_z = round((maxz + minz) / 2, 2)
        size_z = round(abs(maxz - minz) + 3, 2)
        assert (type(cent_x) != None), "Protein structure is damaged"
        assert (type(cent_y) != None), "Protein structure is damaged"
        assert (type(cent_z) != None), "Protein structure is damaged"


        print("Center point of docking grid for {} is as follows: "
              "x: {}, y: {}, z: {}".format(self.protein, size_x, size_y, size_z))
        print("Sizes of docking grid are as follows:"
              "x: {}, y: {}, z: {}".format(cent_x, cent_y, cent_z))
        print("Receptor: ", self.protein, "\n", "Ligand: ", self.ligand, "\n", "Output file: ", self.dockfile)
        os.system(
            'vina --receptor {} --ligand "{}" --center_x {} --center_y {} --center_z {} --size_x {} --size_y {} --size_z {} --log "{}" --out "{}"'.format(
                self.protein,
                self.ligand,
                cent_x,
                cent_y,
                cent_z,
                size_x,
                size_y,
                size_z,
                self.docklog,
                self.dockfile
            ))
        """Postprocessing of docking files"""
        # time.sleep(60)
        df.df['ATOM']['segment_id'].replace(r'.{1,}', '', regex=True, inplace=True) # Clean pdbqt inheritance
        df.df['ATOM']['blank_4'].replace(r'.{1,}', '', regex=True, inplace=True)
        docking_output_df = PandasPdb().read_pdb(self.dockfile)
        docking_output_df.df['HETATM'].drop_duplicates(subset='atom_number', keep='first', inplace=True)  # extract first model
        docking_output_df.df['HETATM']['segment_id'].replace(r'.{1,}', '', regex=True, inplace=True)
        docking_output_df.df['HETATM']['blank_4'].replace(r'.{1,}', '', regex=True, inplace=True)  # clean pdbqt inheritance
        df.df['ATOM'] = df.df['ATOM'].append(docking_output_df.df['HETATM'], ignore_index=True) # merges the files
        df.to_pdb(path = self.complex_name,
                  records=['ATOM','HETATM'],
                  gz = False,
                  append_newline=True)
        # try:
        #     ObtainInteractionsFromComplex(self.complex_name).connect_retrieve() # PLIP analysis
        # except:
        #     pass
        shutil.move(self.complex_name, './results')

In [80]:
#Dummy CID:
# top_cid=['6500', '6501', '6502']

# Part 1 - look for PDB entry
from Bio import SwissProt
import tempfile
handle = ExPASy.get_sprot_raw(check_id)
record = SwissProt.read(handle)
for i in record.cross_references:
    if i[0] == 'PDB':
        entry_pdb = i[1] # Pick the first one
        break
#Part 2 - download and prepare PDB
prepare_protein_path = '~/MGLTools-1.5.6/bin/pythonsh ~/MGLTools-1.5.6/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py -A bonds_hydrogens -U nphs_lps_waters_nonstdres_deleteAltB -r'
with tempfile.TemporaryDirectory(dir=os.getcwd()) as tmpdir:
# tmpdir = os.getcwd()+'/testcase'
    ProteinPreparer(entry_pdb.lower(), tmpdir).prepare_protein()
    #Part 3 - prepare ligands
    for CID_entry in top_cid:
        LigandPreparer(CID_entry,tmpdir).prepare_ligand()
        #Part 4 - dock the ligand
        VinaDocker(protentry=entry_pdb.lower(),
                            ligentry=CID_entry,
                            protein_pdbqt=tmpdir + '/' + entry_pdb.lower(),
                            ligand_pdbqt=tmpdir + '/' + CID_entry,
                            dir=tmpdir
                            ).dock_merge_plip()
    

    








Center point of docking grid for /home/rmadeye/PycharmProjects/target2drug/tmpahrv5bd0/1w77.pdbqt is as follows: x: 49.39, y: 50.04, z: 53.25
Sizes of docking grid are as follows:x: 12.92, y: 64.08, z: -23.53
Receptor:  /home/rmadeye/PycharmProjects/target2drug/tmpahrv5bd0/1w77.pdbqt 
 Ligand:  /home/rmadeye/PycharmProjects/target2drug/tmpahrv5bd0/6500.pdbqt 
 Output file:  1w77_6500.out
Center point of docking grid for /home/rmadeye/PycharmProjects/target2drug/tmpahrv5bd0/1w77.pdbqt is as follows: x: 49.39, y: 50.04, z: 53.25
Sizes of docking grid are as follows:x: 12.92, y: 64.08, z: -23.53
Receptor:  /home/rmadeye/PycharmProjects/target2drug/tmpahrv5bd0/1w77.pdbqt 
 Ligand:  /home/rmadeye/PycharmProjects/target2drug/tmpahrv5bd0/6501.pdbqt 
 Output file:  1w77_6501.out
Center point of docking grid for /home/rmadeye/PycharmProjects/target2drug/tmpahrv5bd0/1w77.pdbqt is as follows: x: 49.39, y: 50.04, z: 53.25
Sizes of docking grid are as follows:x: 12.92, y: 64.08, z: -23.53
Receptor:

In [89]:
#Part 5 - PLIP
#protein-ligand interaction profiling through PLIP 
options=Options()
options.headless = True
plip = webdriver.Firefox(options=options)
plip.get("https://projects.biotec.tu-dresden.de/plip-web/plip")
print("Connection successful")

for cplx in os.listdir('./results/'):
    if cplx.endswith('cplx.pdb'):
        cplx=f'./results/{cplx}'
        print("Processing file {}".format(cplx))
    
        select_pdb_input = plip.find_element_by_xpath("//*[@id='select-pdb-by-file']").click() 

        browse = plip.find_element_by_xpath(
            '/html/body/div[1]/div[2]/div/form/div[1]/div[1]/div[3]/input'
        ).send_keys(                                                       
            os.getcwd()+'/{}'.format(cplx)
        )

        send_file = plip.find_element_by_xpath("//*[@id='submit']").click() 
        time.sleep(10) 
        try:
            try:
                open_interactions_1 = plip.find_element_by_xpath('/html/body/div/div[2]/div/div[1]/h2[2]').click()
                open_interactions_2 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div[2]/h3').click()
                open_interactions_3 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div[2]/div/h4').click()
                pngs = plip.find_elements_by_xpath("//a[contains(@href,'.png')]")
                pymolsessions = plip.find_elements_by_xpath("//a[contains(@href,'.pse')]")
                
            except:
                open_interactions_1 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/h2[1]').click()
                open_interactions_2 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div[1]/h3').click()
                open_interactions_3 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div[1]/div/h4').click()
                pngs = plip.find_elements_by_xpath("//a[contains(@href,'.png')]")
                pymolsessions = plip.find_elements_by_xpath("//a[contains(@href,'.pse')]")

            for image in pngs:
                print(image.get_attribute("href"))
                output_image = requests.get(image.get_attribute("href"))
                open(
                    os.getcwd()+'/{}'.format(cplx+'.png'), 'wb'
                ).write(output_image.content)
                print("Image saved as {}".format(cplx+'.png'))

            for pysession in pymolsessions:
                print(pysession.get_attribute("href"))
                pse = requests.get(pysession.get_attribute("href"))
                open(
                    os.getcwd()+'/{}'.format(cplx+'.pse'), 'wb'
                ).write(pse.content)
                print("Pymol sessions saved as {}".format(cplx+'.pse'))
                  
            restart_plip = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/p[3]/a').click()
            time.sleep(5)
        except:
            print("No interactions found for {} or damaged structure".format(cplx))
            try:
                restart_plip = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/p[3]/a').click()
                time.sleep(5)
            except:
                restart_plip = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/p/a').click() 

Connection successful
Processing file ./results/1w77_6502_cplx.pdb
https://projects.biotec.tu-dresden.de/plip-web/plip/download/cfe86d9c-9bf6-4f96-8bc4-9264b8ac37f7?filePath=outputs%2F1W77_6502_CPLX_PROTEIN_UNL_Z_1.png
Image saved as ./results/1w77_6502_cplx.pdb.png
https://projects.biotec.tu-dresden.de/plip-web/plip/download/cfe86d9c-9bf6-4f96-8bc4-9264b8ac37f7?filePath=outputs%2F1W77_6502_CPLX_PROTEIN_UNL_Z_1.pse
Pymol sessions saved as ./results/1w77_6502_cplx.pdb.pse
Processing file ./results/1w77_6501_cplx.pdb
https://projects.biotec.tu-dresden.de/plip-web/plip/download/670e0f14-ee84-487c-89aa-02ef1f782034?filePath=outputs%2F1W77_6501_CPLX_PROTEIN_UNL_Z_1.png
Image saved as ./results/1w77_6501_cplx.pdb.png
https://projects.biotec.tu-dresden.de/plip-web/plip/download/670e0f14-ee84-487c-89aa-02ef1f782034?filePath=outputs%2F1W77_6501_CPLX_PROTEIN_UNL_Z_1.pse
Pymol sessions saved as ./results/1w77_6501_cplx.pdb.pse
Processing file ./results/1w77_6500_cplx.pdb
https://projects.biotec.t

PROJECT ENDS